In [ ]:
import xarray as xr
import rioxarray as riox
import pandas as pd
import os
import re
import numpy as np
import panel as pn
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
from cartopy import crs
import param as pm
import geopandas as gpd
import glob
from fnmatch import fnmatch
from shapely.geometry import Polygon
import datetime
from tqdm.notebook import tqdm
import shutil
pn.extension()
hv.extension('bokeh')
pn.param.ParamMethod.loading_indicator = True

In [ ]:
"""
Get the server base url
"""
from jupyter_server import serverapp
from jupyter_server.utils import url_path_join
from pathlib import Path
import re
import requests
 
jupServer = [x for x in serverapp.list_running_servers()][0]

In [ ]:
group_training = True
os.chdir('/project/cper_neon_aop/cper_pdog_uas/')
base_dir = os.getcwd()
trainer_dir = 'sean.kearney/'
tmp_dir = os.path.join('/local/bgfs/' + trainer_dir, os.listdir('/local/bgfs/' + trainer_dir)[0])

In [ ]:
polys_dirs = [x.split('/')[-2] for x in glob.glob('train_polys/*_group/')]

In [ ]:
gdf_ground_points_all = gpd.read_file('ground_data/cper_pdog_points_2021Sept.shp')

for idx, pasture in enumerate(['5W', '22W', '22E', '29-30', 'CN']):
    if idx == 0:
        df_bboxes = pd.read_csv('train_tiles/train_bboxes_' + pasture + '.csv')
    else:
        df_bboxes = pd.concat([df_bboxes, pd.read_csv('train_tiles/train_bboxes_' + pasture + '.csv')])

df_bboxes = df_bboxes[df_bboxes['Digitize'] == 1].reset_index(drop=True)
df_bboxes['path_pre'] = df_bboxes.apply(lambda x: os.path.join('train_tiles',
                                                           x['Pasture'] + '_' + x['Type'],
                                                           x['Pasture'] + '_' + x['Tile'] + '_'), axis=1)
df_bboxes.loc[df_bboxes[df_bboxes['Type'] == 'random'].groupby(
    'Pasture').sample(n=2, random_state=123).index, 'trainer'] = 'All'
df_bboxes.loc[df_bboxes[df_bboxes['Type'] == 'burrows_active'].groupby(
    'Pasture').sample(n=1, random_state=223).index, 'trainer'] = 'All'

df_bboxes_sub = df_bboxes.loc[df_bboxes['trainer'] != 'All'].sample(frac=1, random_state=323)

df_bboxes_sub.iloc[:30, df_bboxes_sub.columns.get_loc('trainer')] = 'Lauren'
df_bboxes_sub.iloc[30:60, df_bboxes_sub.columns.get_loc('trainer')] = 'David'
df_bboxes_sub.iloc[60:90, df_bboxes_sub.columns.get_loc('trainer')] = 'Sean'
df_bboxes_sub.iloc[90:, df_bboxes_sub.columns.get_loc('trainer')] = 'Nick'

df_bboxes_fnl = pd.concat([df_bboxes[df_bboxes['trainer'] == 'All'], df_bboxes_sub])
df_bboxes_fnl.loc[df_bboxes_fnl['Poly_ID'].isnull(), 'Poly_ID'] = 'None'
type_dict = {'burrows_active': 0, 'random': 1}

In [ ]:
cur_paths = list(df_bboxes_fnl[df_bboxes_fnl['trainer'] == 'All']['path_pre'].values)

In [ ]:
os.chdir(tmp_dir)

In [ ]:
print('Preparing data on fast local $TMPDIR storage. Link to app will appear when finished.\n\n')
if not os.path.exists('ground_data'):
    os.mkdir('ground_data')
print('   copying ground data...')
for f in tqdm(glob.glob('/project/cper_neon_aop/cper_pdog_uas/ground_data/*.*')):
    shutil.copy(f, 'ground_data/')

if not os.path.exists('train_tiles'):
    os.mkdir('train_tiles')
print('   copying training tile bounding box data...')
for csv in tqdm(glob.glob('/project/cper_neon_aop/cper_pdog_uas/train_tiles/*.csv')):
    shutil.copy(csv, 'train_tiles/')

print('   copying tile data (be patient - this may take a few minutes!)...')
for p in tqdm(cur_paths):
    subdir = os.path.dirname(p).split('/')[-1]
    if not os.path.exists('train_tiles/' + subdir):
        os.mkdir('train_tiles/' + subdir)
    for tif in glob.glob(base_dir + '/' + p + '*.tif'):
        if not os.path.exists('train_tiles/' + subdir + '/' + os.path.basename(tif)):
            shutil.copy(tif, 'train_tiles/' + subdir)

# get existing polygons already saved by all users for group training
polyfiles = []
for poly_dir in polys_dirs:
    polyfiles_tmp = list(filter(os.path.isfile, 
                    glob.glob('/project/cper_neon_aop/cper_pdog_uas/train_polys/' + poly_dir + '/delim_' +\
                                       poly_dir + '_' + '*.shp')))
    if len(polyfiles_tmp) > 0:
        polyfiles_tmp.sort(key=lambda x: os.path.getctime(x))
    polyfiles.append(polyfiles_tmp[-1])
            

In [ ]:
class TrainApp(pm.Parameterized):
    window_size_m = 30
    slider_width=150
    type_dict = {'burrows_active': 0, 'random': 1}
    
    for idx, f in enumerate(polyfiles):
        if idx == 0:
            delim_burrows = gpd.read_file(f)
        else:
            delim_burrows = delim_burrows.append(gpd.read_file(f))
    delim_burrows = delim_burrows.reset_index()
    df_bboxes_all = df_bboxes_fnl.copy(deep=True)
    df_bboxes = df_bboxes_all[df_bboxes_all['trainer'] == 'All'].sort_values('Type',
                                                                             key=lambda x: x.map(type_dict))

    #tile_file_list = glob.glob('train_tiles/' + pasture + '**/*.tif')
    
    gdf_ground_points = gdf_ground_points_all#[gdf_ground_points_all['Pasture'] == pasture]
    gdf_burrows = gdf_ground_points[gdf_ground_points['is_burrow'] == 1]
    gdf_other = gdf_ground_points[gdf_ground_points['is_burrow'] == 0]

    ndvi_alpha = pn.widgets.FloatSlider(start=0.0, end=1.0, step=0.05, value=0.0, width=slider_width, name='Transparency')
    ndvi_range = pn.widgets.RangeSlider(start=0.0, end=0.50, step=0.01, value=(0.0, 0.20), width=slider_width, name='Color range')

    terrain_alpha = pn.widgets.FloatSlider(start=0.0, end=1.0, step=0.05, value=0.0, width=slider_width, name='Transparency')
    tpi_alpha = pn.widgets.FloatSlider(start=0.0, end=0.5, step=0.05, value=0.0, width=slider_width, name='Transparency')
    
    terrain_range = pn.widgets.RangeSlider(start=-0.50, end=1.0, step=0.01, value=(-0.10, 0.40), width=slider_width, name='Color range')   

    burrows_alpha = pn.widgets.FloatSlider(start=0.0, end=1.0, step=0.05, value=0.9, 
                                      width=slider_width, name='Transparency')
    
    ground_alpha = pn.widgets.FloatSlider(start=0.0, end=1.0, step=0.05, value=0.0, 
                                          width=slider_width, name='Transparency')
    
    #tile_list = df_bboxes['ID'].sort_values().to_list()
    
    tile_list = df_bboxes['ID'].to_list()
    tile_list_dict = {k: v for (k, v) in zip(df_bboxes['ID'].to_list(), 
                                             df_bboxes['ID'].index.values)}
    
    #select_tile = pm.Selector(objects=tile_list.param, default=tile_list.param.value[0])
    select_tile = pn.widgets.Select(options=tile_list, name='Select tile', width=slider_width, size=5)
    
    map_opts = dict(projection=crs.UTM(13), responsive=False, width=700, height=700, xaxis=None, yaxis=None,
                     padding=0, tools=['pan', 'box_zoom'], framewise=False,
                     active_tools=['wheel_zoom'], toolbar='left')
    
    map_args = dict(rasterize=True, project=False, dynamic=True)
    
    if len(polyfiles) > 0:
        burrow_poly_dict = {}
        for k in df_bboxes_all['ID']:
            delim_burrows_tmp = delim_burrows[delim_burrows['Tile'] == k]
            if len(delim_burrows_tmp) > 0:
                d_tmp = []
                for i in delim_burrows_tmp.index:
                    d_tmp.append(dict(x=delim_burrows_tmp.geometry[i].exterior.coords.xy[0],
                                      y=delim_burrows_tmp.geometry[i].exterior.coords.xy[1],
                                      Comment=delim_burrows_tmp['Comment'][i],
                                      Tile=delim_burrows_tmp['Tile'][i],
                                      Trainer=delim_burrows_tmp['Trainer'][i]))
                burrow_poly_dict[k] = hv.Polygons(data=d_tmp, vdims='Trainer').opts(line_color='Trainer',
                                                                                    fill_color=None,
                                                                                    fill_alpha=0.0,
                                                                                    line_width=2.0,
                                                                                    cmap='tab10',
                                                                                    tools=['hover'])
            else:
                burrow_poly_dict[k] = hv.Polygons([])
    else:
        burrow_poly_dict = {k: hv.Polygons([]) for k in df_bboxes_all['ID']}
        
    def __init__(self, **params):
        super(TrainApp, self).__init__(**params)
        
        #self.i = int(self.select_tile.value.split('_')[-1])
        #self.select_tile = pn.widgets.Select(options=self.tile_list, name='Select tile', width=self.slider_width, size=5)
        self.i = self.tile_list_dict[self.select_tile.value]
        
        self.tpi_alpha.value = self.terrain_alpha.value / 2.0
        
        #self.tiles = [x for x in self.tile_file_list if fnmatch(x.split('/')[-1], self.pasture + '_' + self.select_tile.value + '_*.tif')]
        self.rgb = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'rgb.tif').astype('int')
        self.rgb_img = self.rgb.hvplot.rgb(x='x', y='y', bands='band',
                                               **self.map_args).opts(**self.map_opts)
        self.rangexy = hv.streams.RangeXY(source=self.rgb_img)


        self.ndvi = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'ndvi.tif').squeeze()
        self.ndvi_img = self.ndvi.hvplot.image(x='x', y='y', alpha=0.5,
                                                         **self.map_args).opts(cmap='viridis',
                                                                               colorbar=False,
                                                                               **self.map_opts)

        self.shade = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'shade.tif').squeeze()
        self.tpi = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'tpi.tif').squeeze()
        
        self.shade_img = self.shade.hvplot.image(x='x', y='y', 
                                     **self.map_args).opts(cmap='gray',
                                                           clim=(210, 225),
                                                           colorbar=False,
                                                           **self.map_opts)
        self.tpi_img = self.tpi.hvplot.image(x='x', y='y',
                                                    **self.map_args).opts(cmap='turbo',
                                                                          colorbar=False,
                                                                          **self.map_opts)

        
        self.minx, self.miny, self.maxx, self.maxy = self.df_bboxes.loc[self.i, ['min_x', 'min_y', 'max_x', 'max_y']].values
        
        self.poly_tmp = hv.Polygons(hv.Bounds((self.minx, self.miny, self.maxx, self.maxy))).opts('Polygons',
                                                                                                  line_color='red',
                                                                                                  fill_color=None, 
                                                                                                  **self.map_opts)
        
        self.p_ground_burrows = hv.Points(data=list(zip(self.gdf_burrows.geometry.x.values,
                                               self.gdf_burrows.geometry.y.values))).opts(color='red', 
                                                                                     marker='o',
                                                                                     size=12,
                                                                                     framewise=False)
        self.p_ground_other = hv.Points(data=list(zip(self.gdf_other.geometry.x.values,
                                                   self.gdf_other.geometry.y.values))).opts(color='orange', 
                                                                                       marker='x',
                                                                                       line_width=4,
                                                                                       size=15,
                                                                                       framewise=False)
        self.p_burrows = self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']]
        
    @pm.depends('terrain_alpha.param', watch=True)
    def calc_tpi_alpha(self):
        self.tpi_alpha.value = self.terrain_alpha.value / 2.0
    
    def show_rgb(self):
        return self.rgb_img
    
    def show_ndvi(self):
        return self.ndvi_img.apply.opts(alpha=self.ndvi_alpha.param.value,
                                        clim=self.ndvi_range.param.value)

    def show_terrain(self):
        return (self.shade_img.apply.opts(alpha=self.terrain_alpha.param.value) * self.tpi_img.apply.opts(alpha=self.tpi_alpha.param.value,
                                                                                                      clim=self.terrain_range.param.value))
    
    @pm.depends('select_tile.param')
    def view(self):            
        
        #self.i = int(self.select_tile.value.split('_')[-1])
        self.i = self.tile_list_dict[self.select_tile.value]
        
        self.p_burrows = self.burrow_poly_dict[self.df_bboxes.loc[self.i, 'ID']]

        self.rgb = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'rgb.tif').astype('int')
        self.rgb_img = self.rgb.hvplot.rgb(x='x', y='y', bands='band',
                                               **self.map_args).opts(**self.map_opts)

        self.ndvi = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'ndvi.tif').squeeze()
        self.ndvi_img = self.ndvi.hvplot.image(x='x', y='y',
                                                         **self.map_args).opts(cmap='viridis',
                                                                               colorbar=False,
                                                                               **self.map_opts)

        self.shade = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'shade.tif').squeeze()
        self.tpi = riox.open_rasterio(self.df_bboxes.loc[self.i, 'path_pre'] + 'tpi.tif').squeeze()
        self.shade_img = self.shade.hvplot.image(x='x', y='y', 
                                     **self.map_args).opts(cmap='gray',
                                                           clim=(210, 225),
                                                           colorbar=False,
                                                           **self.map_opts)
        self.tpi_img = self.tpi.hvplot.image(x='x', y='y',
                                                    **self.map_args).opts(cmap='turbo',
                                                                          colorbar=False,
                                                                          **self.map_opts)

        
        self.minx, self.miny, self.maxx, self.maxy = self.df_bboxes.loc[self.i, ['min_x', 'min_y', 'max_x', 'max_y']].values
        self.poly_tmp = hv.Polygons(hv.Bounds((self.minx, self.miny, self.maxx, self.maxy))).opts('Polygons',
                                                                                                  line_color='red',
                                                                                                  fill_color=None, 
                                                                                                  **self.map_opts)        
        if self.df_bboxes.loc[self.i]['Type'] == 'burrows_active':
            return pn.Column(hv.annotate.compose(hv.Overlay([self.rgb_img,
                                       self.ndvi_img.apply.opts(alpha=self.ndvi_alpha.param.value,
                                                                clim=self.ndvi_range.param.value), 
                                       (self.shade_img.apply.opts(alpha=self.terrain_alpha.param.value) * self.tpi_img.apply.opts(alpha=self.tpi_alpha.param.value,
                                                                                                          clim=self.terrain_range.param.value)),
                                       self.poly_tmp]).collate(),
                                       self.p_ground_burrows.apply.opts(alpha=self.ground_alpha.param.value),
                                       self.p_ground_other.apply.opts(alpha=self.ground_alpha.param.value),
                                       self.p_burrows.apply.opts(line_alpha=self.burrows_alpha.param.value)).opts(toolbar='left').redim.range(x=(np.min(self.rgb['x']) + self.window_size_m*0.75,
                                                                                                np.max(self.rgb['x']) - self.window_size_m*0.75),
                                                                                             y=(np.min(self.rgb['y']) + self.window_size_m*0.75,
                                                                                                np.max(self.rgb['y']) - self.window_size_m*0.75)))
        else:
            return pn.Column(hv.annotate.compose(hv.Overlay([self.rgb_img,
                                       self.ndvi_img.apply.opts(alpha=self.ndvi_alpha.param.value,
                                                                clim=self.ndvi_range.param.value), 
                                       (self.shade_img.apply.opts(alpha=self.terrain_alpha.param.value) * self.tpi_img.apply.opts(alpha=self.tpi_alpha.param.value,
                                                                                                          clim=self.terrain_range.param.value)),
                                       self.poly_tmp]).collate(),
                                       self.p_burrows.apply.opts(line_alpha=self.burrows_alpha.param.value)).opts(toolbar='left').redim.range(x=(np.min(self.rgb['x']) + self.window_size_m*0.75,
                                                                                                np.max(self.rgb['x']) - self.window_size_m*0.75),
                                                                                             y=(np.min(self.rgb['y']) + self.window_size_m*0.75,
                                                                                                np.max(self.rgb['y']) - self.window_size_m*0.75)))
    
    def layout(self):
        return pn.Row(
            pn.Column(
                pn.Column('### Update data', 
                          self.select_tile,                                                       
                          background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                pn.Column(
                    pn.Column('### NDVI', self.ndvi_alpha, self.ndvi_range, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                    pn.Column('### Terrain', self.terrain_alpha, self.terrain_range, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                    pn.Column('### Ground data', self.ground_alpha, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50),
                    pn.Column('### Train data', self.burrows_alpha, background='WhiteSmoke', margin=(5, 5, 5, 5), width=self.slider_width+50)
                ), background='grey'),
            self.view)

print('\n\n\nAll data copied. Launching app....')
app = TrainApp()

port = 3332

app_served = pn.serve(app.layout().servable(),
                      websocket_origin='ceres-ood.scinet.usda.gov', port=port, show=False, verbose=False)

#url = 'https://jupyterhub.scinet.usda.gov/user/' + trainer_dir + 'proxy/' + str(port) + '/'
url = 'https://ceres-ood.scinet.usda.gov' + jupServer['base_url'] + 'proxy/' + str(port) + '/'
print('\n\nApp launched and ready! Click the link below to run the app: \n\n' + url)

In [ ]:
#url = 'https://ceres-ood.scinet.usda.gov' + jupServer['base_url'] + 'proxy/' + str(port) + '/'
#print('\n\nApp launched and ready! Click the link below to run the app: \n\n' + url)

In [ ]:
#app_served.stop()

In [ ]:
#app.layout()#.servable()

In [ ]:
#getNotebookPath()

In [ ]:
#for jupServ in serverapp.list_running_servers():
#    print(jupServ)